## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import dependencies

# from citipy import citipy
# from datetime import datetime
import gmaps
# import matplotlib.pyplot as plt
# import numpy as np
import os
import pandas as pd
import requests
# import time

# Import API keys from config.py file (one level up)
import sys
sys.path.insert(0, '../')
from config import weather_api_key, g_key

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_path = os.path.join('..','Weather_Database','WeatherPy_Database.csv')

df_city_data = pd.read_csv(city_data_path)
df_city_data = df_city_data.set_index('City_ID')
df_city_data.index.name = None

df_city_data.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Salaga,GH,8.5508,-0.5188,80.42,74,93,5.99,overcast clouds
1,Gizo,SB,-8.1030,156.8419,83.80,68,17,4.76,few clouds
2,Punta Arenas,CL,-53.1500,-70.9167,42.91,75,0,8.05,clear sky
3,Rikitea,PF,-23.1203,-134.9692,77.13,71,33,18.75,scattered clouds
4,Shasta Lake,US,40.6804,-122.3708,88.52,37,96,7.02,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
temp_min = float(input("What is the lowest maximum temperature you would like for your trip? "))
temp_max = float(input("What is the highest maximum temperature you would like for your trip? "))

What is the lowest maximum temperature you would like for your trip? 75
What is the highest maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
df_preferred_cities = df_city_data.loc[(temp_min <= df_city_data['Max Temp']) & (df_city_data['Max Temp'] <= temp_max)]

df_preferred_cities.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Salaga,GH,8.5508,-0.5188,80.42,74,93,5.99,overcast clouds
1,Gizo,SB,-8.1030,156.8419,83.80,68,17,4.76,few clouds
3,Rikitea,PF,-23.1203,-134.9692,77.13,71,33,18.75,scattered clouds
4,Shasta Lake,US,40.6804,-122.3708,88.52,37,96,7.02,overcast clouds
16,Kununurra,AU,-15.7667,128.7333,89.58,51,0,5.75,clear sky
19,Yeppoon,AU,-23.1333,150.7333,78.58,75,25,8.37,scattered clouds
20,Atar,MR,20.5169,-13.0499,88.70,17,100,9.33,overcast clouds
23,Georgetown,MY,5.4112,100.3354,80.53,93,20,7.00,few clouds
24,Avarua,CK,-21.2078,-159.7750,78.85,65,75,3.44,broken clouds
26,Cockburn Town,TC,21.4612,-71.1419,78.89,83,100,20.94,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
df_preferred_cities.isna().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if sum(df_preferred_cities.isna().sum()) != 0:  # If there are no rows with empty values, don't bother
    df_preferred_cities = df_preferred_cities.dropna()

df_preferred_cities.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Salaga,GH,8.5508,-0.5188,80.42,74,93,5.99,overcast clouds
1,Gizo,SB,-8.1030,156.8419,83.80,68,17,4.76,few clouds
3,Rikitea,PF,-23.1203,-134.9692,77.13,71,33,18.75,scattered clouds
4,Shasta Lake,US,40.6804,-122.3708,88.52,37,96,7.02,overcast clouds
16,Kununurra,AU,-15.7667,128.7333,89.58,51,0,5.75,clear sky
19,Yeppoon,AU,-23.1333,150.7333,78.58,75,25,8.37,scattered clouds
20,Atar,MR,20.5169,-13.0499,88.70,17,100,9.33,overcast clouds
23,Georgetown,MY,5.4112,100.3354,80.53,93,20,7.00,few clouds
24,Avarua,CK,-21.2078,-159.7750,78.85,65,75,3.44,broken clouds
26,Cockburn Town,TC,21.4612,-71.1419,78.89,83,100,20.94,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = df_preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Salaga,GH,80.42,overcast clouds,8.5508,-0.5188,
1,Gizo,SB,83.80,few clouds,-8.1030,156.8419,
3,Rikitea,PF,77.13,scattered clouds,-23.1203,-134.9692,
4,Shasta Lake,US,88.52,overcast clouds,40.6804,-122.3708,
16,Kununurra,AU,89.58,clear sky,-15.7667,128.7333,
19,Yeppoon,AU,78.58,scattered clouds,-23.1333,150.7333,
20,Atar,MR,88.70,overcast clouds,20.5169,-13.0499,
23,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
24,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
26,Cockburn Town,TC,78.89,overcast clouds,21.4612,-71.1419,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": g_key
# }
input_parameters = {
    'radius': 5000,  # meters
    'type': 'lodging',  # See https://developers.google.com/maps/documentation/places/web-service/supported_types
    'key': g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    input_parameters['location'] = f'{lat},{lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=input_parameters).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except IndexError:
        # With no API key, they will all be errors
        if g_key == 'your Google API key goes here':
            hotel_df.loc[index,'Hotel Name'] = 'need g_key'
        print(f"Hotel near {row['City']}, {row['Country']} not found… skipping.")

Hotel near Salaga, GH not found… skipping.
Hotel near Gizo, SB not found… skipping.
Hotel near Rikitea, PF not found… skipping.
Hotel near Shasta Lake, US not found… skipping.
Hotel near Kununurra, AU not found… skipping.
Hotel near Yeppoon, AU not found… skipping.
Hotel near Atar, MR not found… skipping.
Hotel near Georgetown, MY not found… skipping.
Hotel near Avarua, CK not found… skipping.
Hotel near Cockburn Town, TC not found… skipping.
Hotel near San Andres, CO not found… skipping.
Hotel near Atuona, PF not found… skipping.
Hotel near Butaritari, KI not found… skipping.
Hotel near Tagusao, PH not found… skipping.
Hotel near Kalmunai, LK not found… skipping.
Hotel near Ocos, GT not found… skipping.
Hotel near Vaini, TO not found… skipping.
Hotel near Pochutla, MX not found… skipping.
Hotel near Mersing, MY not found… skipping.
Hotel near Gatesville, US not found… skipping.
Hotel near Tayu, ID not found… skipping.
Hotel near Cabo San Lucas, MX not found… skipping.
Hotel near Safag

Hotel near Cairns, AU not found… skipping.
Hotel near Saint Pete Beach, US not found… skipping.
Hotel near Fare, PF not found… skipping.
Hotel near Belmonte, BR not found… skipping.
Hotel near Islamabad, PK not found… skipping.
Hotel near Dakar, SN not found… skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Salaga,GH,80.42,overcast clouds,8.5508,-0.5188,need g_key
1,Gizo,SB,83.80,few clouds,-8.1030,156.8419,need g_key
3,Rikitea,PF,77.13,scattered clouds,-23.1203,-134.9692,need g_key
4,Shasta Lake,US,88.52,overcast clouds,40.6804,-122.3708,need g_key
16,Kununurra,AU,89.58,clear sky,-15.7667,128.7333,need g_key
19,Yeppoon,AU,78.58,scattered clouds,-23.1333,150.7333,need g_key
20,Atar,MR,88.70,overcast clouds,20.5169,-13.0499,need g_key
23,Georgetown,MY,80.53,few clouds,5.4112,100.3354,need g_key
24,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,need g_key
26,Cockburn Town,TC,78.89,overcast clouds,21.4612,-71.1419,need g_key


In [10]:
# 8a. Create the output File (CSV)
output_filepath = os.path.join('.','WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_filepath, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
fig = gmaps.figure(
    center=(30.0,31.0),
    zoom_level=1.5
)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))